In [1]:
from pathlib import Path
import cv2
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import math
# import catboost as ctb
from sklearn.metrics import confusion_matrix
from sklearn.metrics import accuracy_score, recall_score, precision_score, f1_score

In [109]:
X = {}
y = {}
for game in range(1, 11):
    paths = sorted(list(Path(f'../Dataset/Dataset/game{game}').glob(f'*/*.csv')), key = lambda tmp: (int(str(tmp).split('\\')[3].split('e')[1]), int(str(tmp).split('\\')[4].split('p')[1])))
    labels = pd.read_csv(str(paths[0]))
    num = 3
    eps = 1e-15
#     direction = math.atan2(dy, dx) * 180 / math.pi
    for i in range(1, num):
        labels['x_lag_{}'.format(i)] = labels['x-coordinate'].shift(i)
        labels['x_lag_inv_{}'.format(i)] = labels['x-coordinate'].shift(-i)
        labels['y_lag_{}'.format(i)] = labels['y-coordinate'].shift(i)
        labels['y_lag_inv_{}'.format(i)] = labels['y-coordinate'].shift(-i) 
        labels['x_diff_{}'.format(i)] = labels['x_lag_{}'.format(i)] - labels['x-coordinate']
        labels['y_diff_{}'.format(i)] = labels['y_lag_{}'.format(i)] - labels['y-coordinate']
        labels['direction_{}'.format(i)] = np.arctan2(labels['y_diff_{}'.format(i)], labels['x_diff_{}'.format(i)]) * 180 / math.pi
        labels.loc[labels['direction_{}'.format(i)] < 0, 'direction_{}'.format(i)] += 360
        labels['direction_diff_{}'.format(i)] = labels['direction_{}'.format(i)] - labels['direction_{}'.format(i)].shift(1)
        labels['x_diff_inv_{}'.format(i)] = labels['x_lag_inv_{}'.format(i)] - labels['x-coordinate']
        labels['y_diff_inv_{}'.format(i)] = labels['y_lag_inv_{}'.format(i)] - labels['y-coordinate']
        labels['direction_inv_{}'.format(i)] = np.arctan2(labels['y_diff_inv_{}'.format(i)], labels['x_diff_inv_{}'.format(i)]) * 180 / math.pi
        labels.loc[labels['direction_inv_{}'.format(i)] < 0, 'direction_inv_{}'.format(i)] += 360
        labels['direction_diff_inv_{}'.format(i)] = labels['direction_inv_{}'.format(i)] - labels['direction_inv_{}'.format(i)].shift(1)
        labels['x_div_{}'.format(i)] = labels['x_diff_{}'.format(i)]/(labels['x_diff_inv_{}'.format(i)] + eps)
        labels['y_div_{}'.format(i)] = labels['y_diff_{}'.format(i)]/(labels['y_diff_inv_{}'.format(i)] + eps)
    for i in range(1, num):
        labels = labels[labels['x_lag_{}'.format(i)].notna()]
        labels = labels[labels['x_lag_inv_{}'.format(i)].notna()]
    labels = labels[labels['x-coordinate'].notna()] 

    colnames_x = ['x_diff_{}'.format(i) for i in range(1, num)] + \
                 ['x_diff_inv_{}'.format(i) for i in range(1, num)] + \
                 ['x_div_{}'.format(i) for i in range(1, num)] 
    colnames_y = ['y_diff_{}'.format(i) for i in range(1, num)] + \
                 ['y_diff_inv_{}'.format(i) for i in range(1, num)] + \
                 ['y_div_{}'.format(i) for i in range(1, num)]
    colnames_direction = ['direction_{}'.format(i) for i in range(1, num)] + \
                         ['direction_diff_{}'.format(i) for i in range(1, num)] + \
                         ['direction_inv_{}'.format(i) for i in range(1, num)] + \
                         ['direction_diff_inv_{}'.format(i) for i in range(1, num)]
    colnames = colnames_x + colnames_y + colnames_direction

    X_train = labels[colnames]
    y_train = labels['status'] > 1

    for path in paths[1:]:
        labels = pd.read_csv(str(path))
        num = 3
        eps = 1e-15
        for i in range(1, num):
            labels['x_lag_{}'.format(i)] = labels['x-coordinate'].shift(i)
            labels['x_lag_inv_{}'.format(i)] = labels['x-coordinate'].shift(-i)
            labels['y_lag_{}'.format(i)] = labels['y-coordinate'].shift(i)
            labels['y_lag_inv_{}'.format(i)] = labels['y-coordinate'].shift(-i) 
            labels['x_diff_{}'.format(i)] = labels['x_lag_{}'.format(i)] - labels['x-coordinate']
            labels['y_diff_{}'.format(i)] = labels['y_lag_{}'.format(i)] - labels['y-coordinate']
            labels['direction_{}'.format(i)] = np.arctan2(labels['y_diff_{}'.format(i)], labels['x_diff_{}'.format(i)]) * 180 / math.pi
            labels.loc[labels['direction_{}'.format(i)] < 0, 'direction_{}'.format(i)] += 360
            labels['direction_diff_{}'.format(i)] = labels['direction_{}'.format(i)] - labels['direction_{}'.format(i)].shift(1)
            labels['x_diff_inv_{}'.format(i)] = labels['x_lag_inv_{}'.format(i)] - labels['x-coordinate']
            labels['y_diff_inv_{}'.format(i)] = labels['y_lag_inv_{}'.format(i)] - labels['y-coordinate']
            labels['direction_inv_{}'.format(i)] = np.arctan2(labels['y_diff_inv_{}'.format(i)], labels['x_diff_inv_{}'.format(i)]) * 180 / math.pi
            labels.loc[labels['direction_inv_{}'.format(i)] < 0, 'direction_inv_{}'.format(i)] += 360
            labels['direction_diff_inv_{}'.format(i)] = labels['direction_inv_{}'.format(i)] - labels['direction_inv_{}'.format(i)].shift(1)
            labels['x_div_{}'.format(i)] = labels['x_diff_{}'.format(i)]/(labels['x_diff_inv_{}'.format(i)] + eps)
            labels['y_div_{}'.format(i)] = labels['y_diff_{}'.format(i)]/(labels['y_diff_inv_{}'.format(i)] + eps)
        for i in range(1, num):
            labels = labels[labels['x_lag_{}'.format(i)].notna()]
            labels = labels[labels['x_lag_inv_{}'.format(i)].notna()]
        labels = labels[labels['x-coordinate'].notna()] 

        colnames_x = ['x_diff_{}'.format(i) for i in range(1, num)] + \
                     ['x_diff_inv_{}'.format(i) for i in range(1, num)] + \
                     ['x_div_{}'.format(i) for i in range(1, num)]
        colnames_y = ['y_diff_{}'.format(i) for i in range(1, num)] + \
                     ['y_diff_inv_{}'.format(i) for i in range(1, num)] + \
                     ['y_div_{}'.format(i) for i in range(1, num)]
        colnames_direction = ['direction_{}'.format(i) for i in range(1, num)] + \
                         ['direction_diff_{}'.format(i) for i in range(1, num)] + \
                         ['direction_inv_{}'.format(i) for i in range(1, num)] + \
                         ['direction_diff_inv_{}'.format(i) for i in range(1, num)] 
        colnames = colnames_x + colnames_y + colnames_direction

        display(labels[['x-coordinate', 'y-coordinate', 'status'] + colnames])
        features = labels[colnames]
        labels[['x-coordinate', 'y-coordinate', 'status'] + colnames].to_csv('features.csv', index = False)
        break
        new_status = labels['status'] > 1

        X_train = pd.concat([X_train, features], axis = 0)
        y_train = pd.concat([y_train, new_status])
    X[game] = X_train
    y[game] = y_train
    break

,x-coordinate,y-coordinate,status,x_diff_1,x_diff_2,x_diff_inv_1,x_diff_inv_2,x_div_1,x_div_2,y_diff_1,...,y_div_1,y_div_2,direction_1,direction_2,direction_diff_1,direction_diff_2,direction_inv_1,direction_inv_2,direction_diff_inv_1,direction_diff_inv_2
2,724,447,0,-3.0,-3.0,1.0,3.0,-3.000000e+00,-1.000000e+00,9.0,...,-1.000000,-0.541667,108.434949,102.994617,18.434949,NaN,276.340192,277.125016,-12.094757,-5.403791
3,725,438,0,-1.0,-4.0,2.0,1.0,-5.000000e-01,-4.000000e+00,9.0,...,-0.600000,-0.562500,96.340192,102.528808,-12.094757,-0.465809,277.594643,271.789911,1.254452,-5.335106
4,727,423,0,-2.0,-3.0,-1.0,-2.0,2.000000e+00,1.500000e+00,15.0,...,-0.882353,-0.666667,97.594643,97.125016,1.254452,-5.403791,266.633539,266.820170,-10.961104,-4.969741
5,726,406,0,1.0,-1.0,-1.0,-1.0,-1.000000e+00,1.000000e+00,17.0,...,-0.894737,-0.888889,86.633539,91.789911,-10.961104,-5.335106,266.987212,268.408860,0.353673,1.588690
6,725,387,0,1.0,2.0,0.0,-2.0,1.000000e+15,-1.000000e+00,19.0,...,-1.117647,-1.058824,86.987212,86.820170,0.353673,-4.969741,270.000000,266.633539,3.012788,-1.775320
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
123,761,126,0,3.0,4.0,-3.0,-3.0,-1.000000e+00,-1.333333e+00,10.0,...,-1.428571,-0.428571,73.300756,56.309932,-210.735488,-227.726311,246.801409,257.905243,-6.499346,7.345278
124,758,119,0,3.0,6.0,0.0,0.0,3.000000e+15,6.000000e+15,7.0,...,-1.000000,-1.133333,66.801409,70.559965,-6.499346,14.250033,270.000000,270.000000,23.198591,12.094757
125,758,112,0,0.0,3.0,0.0,-1.0,0.000000e+00,-3.000000e+00,7.0,...,-0.875000,-1.272727,90.000000,77.905243,23.198591,7.345278,270.000000,264.805571,0.000000,-5.194429
126,758,104,0,0.0,0.0,-1.0,-3.0,-0.000000e+00,-0.000000e+00,8.0,...,-2.666667,-1.875000,90.000000,90.000000,0.000000,12.094757,251.565051,249.443955,-18.434949,-15.361616


In [86]:
X[1][['x_diff_2', 'y_diff_2', 'direction_2', 'direction_diff_2', 'x_diff_inv_2', 'y_diff_inv_2', 'direction_inv_2', 'direction_diff_inv_2']]

,x_diff_2,y_diff_2,direction_2,direction_diff_2,x_diff_inv_2,y_diff_inv_2,direction_inv_2,direction_diff_inv_2
2,-2.0,35.0,356.729512,NaN,-3.0,-35.0,184.899092,0.000000
3,3.0,35.0,4.899092,-351.830420,-1.0,-34.0,181.684684,-3.214408
4,3.0,35.0,4.899092,0.000000,-2.0,-30.0,183.814075,2.129391
5,1.0,34.0,1.684684,-3.214408,-2.0,-27.0,184.236395,0.422320
6,2.0,30.0,3.814075,2.129391,-2.0,-24.0,184.763642,0.527247
...,...,...,...,...,...,...,...,...
246,-24.0,53.0,335.637547,-1.668658,19.0,-42.0,155.658910,-2.745779
247,-19.0,48.0,338.404690,2.767143,20.0,-36.0,150.945396,-4.713514
248,-19.0,42.0,335.658910,-2.745779,17.0,-34.0,153.434949,2.489553
249,-20.0,36.0,330.945396,-4.713514,15.0,-27.0,150.945396,-2.489553


In [ ]:
import catboost as ctb
import pandas as pd
import numpy as np
from scipy.interpolate import CubicSpline
from scipy.spatial import distance
import math

class BounceDetector:
    def __init__(self, path_model=None):
        self.model = ctb.CatBoostRegressor()
        self.threshold = 0.45
        if path_model:
            self.load_model(path_model)
        
    def load_model(self, path_model):
        self.model.load_model(path_model)
    
    def prepare_features(self, x_ball, y_ball):
        labels = pd.DataFrame({'frame': range(len(x_ball)), 'x-coordinate': x_ball, 'y-coordinate': y_ball})
        
        num = 3
        eps = 1e-15
        for i in range(1, num):
            labels['x_lag_{}'.format(i)] = labels['x-coordinate'].shift(i)
            labels['x_lag_inv_{}'.format(i)] = labels['x-coordinate'].shift(-i)
            labels['y_lag_{}'.format(i)] = labels['y-coordinate'].shift(i)
            labels['y_lag_inv_{}'.format(i)] = labels['y-coordinate'].shift(-i) 
            labels['x_diff_{}'.format(i)] = labels['x_lag_{}'.format(i)] - labels['x-coordinate']
            labels['y_diff_{}'.format(i)] = labels['y_lag_{}'.format(i)] - labels['y-coordinate']
            labels['direction_{}'.format(i)] = np.arctan2(labels['y_diff_{}'.format(i)], labels['x_diff_{}'.format(i)]) * 180 / math.pi
            labels.loc[labels['direction_{}'.format(i)] < 0, 'direction_{}'.format(i)] += 360
            labels['direction_diff_{}'.format(i)] = labels['direction_{}'.format(i)] - labels['direction_{}'.format(i)].shift(1)
            labels['x_diff_inv_{}'.format(i)] = labels['x_lag_inv_{}'.format(i)] - labels['x-coordinate']
            labels['y_diff_inv_{}'.format(i)] = labels['y_lag_inv_{}'.format(i)] - labels['y-coordinate']
            labels['direction_inv_{}'.format(i)] = np.arctan2(labels['y_diff_inv_{}'.format(i)], labels['x_diff_inv_{}'.format(i)]) * 180 / math.pi
            labels.loc[labels['direction_inv_{}'.format(i)] < 0, 'direction_inv_{}'.format(i)] += 360
            labels['direction_diff_inv_{}'.format(i)] = labels['direction_inv_{}'.format(i)] - labels['direction_inv_{}'.format(i)].shift(1)
            labels['x_div_{}'.format(i)] = labels['x_diff_{}'.format(i)]/(labels['x_diff_inv_{}'.format(i)] + eps)
            labels['y_div_{}'.format(i)] = labels['y_diff_{}'.format(i)]/(labels['y_diff_inv_{}'.format(i)] + eps)

        for i in range(1, num):
            labels = labels[labels['x_lag_{}'.format(i)].notna()]
            labels = labels[labels['x_lag_inv_{}'.format(i)].notna()]
        labels = labels[labels['x-coordinate'].notna()] 
        
        colnames_x = ['x_diff_{}'.format(i) for i in range(1, num)] + \
                     ['x_diff_inv_{}'.format(i) for i in range(1, num)] + \
                     ['x_div_{}'.format(i) for i in range(1, num)]
        colnames_y = ['y_diff_{}'.format(i) for i in range(1, num)] + \
                     ['y_diff_inv_{}'.format(i) for i in range(1, num)] + \
                     ['y_div_{}'.format(i) for i in range(1, num)]
        colnames_direction = ['direction_{}'.format(i) for i in range(1, num)] + \
                         ['direction_diff_{}'.format(i) for i in range(1, num)] + \
                         ['direction_inv_{}'.format(i) for i in range(1, num)] + \
                         ['direction_diff_inv_{}'.format(i) for i in range(1, num)] 
        colnames = colnames_x + colnames_y + colnames_direction

        features = labels[colnames]
        return features, list(labels['frame'])
    
    def predict(self, x_ball, y_ball, smooth=True):
        if smooth:
            x_ball, y_ball = self.smooth_predictions(x_ball, y_ball)
        features, num_frames = self.prepare_features(x_ball, y_ball)
        preds = self.model.predict(features)
        ind_bounce = np.where(preds > self.threshold)[0]
        if len(ind_bounce) > 0:
            ind_bounce = self.postprocess(ind_bounce, preds)
        frames_bounce = [num_frames[x] for x in ind_bounce]
        return set(frames_bounce)
    
    def smooth_predictions(self, x_ball, y_ball):
        is_none = [int(x is None) for x in x_ball]
        interp = 5
        counter = 0
        for num in range(interp, len(x_ball)-1):
            if not x_ball[num] and sum(is_none[num-interp:num]) == 0 and counter < 3:
                x_ext, y_ext = self.extrapolate(x_ball[num-interp:num], y_ball[num-interp:num])
                x_ball[num] = x_ext
                y_ball[num] = y_ext
                is_none[num] = 0
                if x_ball[num+1]:
                    dist = distance.euclidean((x_ext, y_ext), (x_ball[num+1], y_ball[num+1]))
                    if dist > 80:
                        x_ball[num+1], y_ball[num+1], is_none[num+1] = None, None, 1
                counter += 1
            else:
                counter = 0  
        return x_ball, y_ball

    def extrapolate(self, x_coords, y_coords):
        xs = list(range(len(x_coords)))
        func_x = CubicSpline(xs, x_coords, bc_type='natural')
        x_ext = func_x(len(x_coords))
        func_y = CubicSpline(xs, y_coords, bc_type='natural')
        y_ext = func_y(len(x_coords))
        return float(x_ext), float(y_ext)    

    def postprocess(self, ind_bounce, preds):
        ind_bounce_filtered = [ind_bounce[0]]
        for i in range(1, len(ind_bounce)):
            if (ind_bounce[i] - ind_bounce[i-1]) != 1:
                cur_ind = ind_bounce[i]
                ind_bounce_filtered.append(cur_ind)
            elif preds[ind_bounce[i]] > preds[ind_bounce[i-1]]:
                ind_bounce_filtered[-1] = ind_bounce[i]
        return ind_bounce_filtered


